## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [ ]:
# %pip install tensorflow keras
# %pip install tqdm
# %pip install tensorflow_text
#%pip install tensorflow==2.19.0 tensorflow-text==2.19.0

from data_parser import xml_parser

import io
import re
import string
import tqdm

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text

import tensorflow as tf
from tensorflow.keras import layers

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


2026-01-08 19:02:43.534168: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-08 19:02:43.538588: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-08 19:02:43.550865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767895363.571164   90419 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767895363.577056   90419 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767895363.595116   90419 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

## Tokenizer

De eerste stap is het omzetten van de text in tokens:


In [ ]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [277]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [278]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [279]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:


In [280]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


In [281]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


In [ ]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([ 3  0  6 10], shape=(4,), dtype=int64)
['is', '<pad>', 'om', 'testen.']


In [283]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


In [284]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")


target_index    : 3
target_word     : is
context_indices : [ 4  3  0  6 10]
context_words   : ['een', 'is', '<pad>', 'om', 'testen.']
label           : [1 0 0 0 0]


In [285]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)


[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [286]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0,
          seed=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=0,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels


## Word2Vec:


In [287]:
text_ds = tf.data.TextLineDataset(["trainingsData.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))


In [288]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
sequence_length = 100 
vocab_size = 4096

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [289]:
vectorize_layer.adapt(text_ds.batch(1024))


In [290]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])


['', '[UNK]', np.str_('the'), np.str_('of'), np.str_('and'), np.str_('in'), np.str_('to'), np.str_('a'), np.str_('for'), np.str_('with'), np.str_('were'), np.str_('was'), np.str_('that'), np.str_('is'), np.str_('cancer'), np.str_('by'), np.str_('as'), np.str_('cells'), np.str_('or'), np.str_('from')]


In [291]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()


In [292]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))


219


In [293]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")


[  81   33  826    3 1147  195   74    1   86   14  210 3230   40   47
    4    1  218    2    1  980    3    2  215 1645 3230   40    2   26
 2081    4  103 1783 2886   39   44   38 1408   12   26 2524   13    1
 1783 2886    1    1    1    1  129  103 1783 2886    1    3    1  237
    1    7  359 2993    3 1260   12    1 1447   17 1586 1147    4  442
   27  315  761   27 1144    4  719  880    6    1  120    1  215    3
   32 2276    1 1100   42   21 3317  564    5    2   26    1    4 1395
    8   26] => [np.str_('gene'), np.str_('expression'), np.str_('signature'), np.str_('of'), np.str_('fibroblast'), np.str_('serum'), np.str_('response'), '[UNK]', np.str_('human'), np.str_('cancer'), np.str_('progression'), np.str_('similarities'), np.str_('between'), np.str_('tumors'), np.str_('and'), '[UNK]', np.str_('since'), np.str_('the'), '[UNK]', np.str_('observations'), np.str_('of'), np.str_('the'), np.str_('many'), np.str_('histologic'), np.str_('similarities'), np.str_('between'), np.st

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=42)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 219/219 [00:03<00:00, 67.35it/s]




targets.shape: (10309,)
contexts.shape: (10309, 5)
labels.shape: (10309, 5)


In [ ]:
BATCH_SIZE = 20
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(20,), dtype=tf.int64, name=None), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None))>


In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [305]:
word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9844 - loss: 0.1236
Epoch 2/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9853 - loss: 0.1056
Epoch 3/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9861 - loss: 0.0924
Epoch 4/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9866 - loss: 0.0827
Epoch 5/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9866 - loss: 0.0753
Epoch 6/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0698
Epoch 7/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9868 - loss: 0.0655
Epoch 8/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0621
Epoch 9/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0596
Epoch 10/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9867 - loss: 0.0575
Epoch 11/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0559
Epoch 12/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [308]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass


## Data:
Als trainingsdata gebruik ik ~3000 Pubmed artikelen die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.filelist.txt


Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan. \

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn classe: data_parser.py



In [3]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/", "/run/media/floris/FLORIS_2/data/trainingsData.txt", keyword_list)

trainings_data.run()



122576 XML bestanden...


 38%|███▊      | 47156/122576 [25:50<52:03, 24.15it/s]  

Error /run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/: 'NoneType' object has no attribute 'get_text'


 51%|█████▏    | 62919/122576 [36:46<20:15, 49.06it/s]  

Error /run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/: 'NoneType' object has no attribute 'get_text'


 53%|█████▎    | 65450/122576 [38:17<33:25, 28.49it/s]  


KeyboardInterrupt: 